# Aim

Inverse detection efficiency completeness calculation on TESS data.

IDEM as in Appendix A of Hsu 2018 (https://arxiv.org/pdf/1803.10787.pdf).

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import sys
sys.path.append('..')
from tqdm.notebook import tqdm
from os import path
from dev import utils

%load_ext autoreload
%autoreload 2

In [6]:
path.exists("../data/tesstargets/TESS_targets_S001.csv")

True

In [3]:
stellar = utils.get_tess_stellar().drop_duplicates("ID")

data/tesstargets/TESS_targets_S001.csv
Getting sector 1 observed targets from https://tess.mit.edu/wp-content/uploads/all_targets_S001_v1.csv.
Querying MAST for sector 1 observed targets.


KeyboardInterrupt: 

In [ ]:
print([k for k in stellar.keys()], end=" ")

In [ ]:
stellar

In [ ]:
planetary = utils.get_tois()

In [ ]:
planetary = planetary[np.isfinite(planetary['koi_prad'])]

In [ ]:
# let's group all these planets into buckets based on planet size and orbital period!
radii = np.array(planetary['koi_prad'])
plt.hist(np.log(radii), bins=100)

In [ ]:
periods = np.array(planetary['koi_period'])
plt.hist(np.log(periods), bins=100)

In [ ]:
plt.plot(np.log(periods), np.log(radii), ".k")
plt.xlabel("Log-period (ln days)")
plt.ylabel("Log-radius (ln Earth radii)")

In [ ]:
planetary = planetary[planetary['koi_disposition'] != "FALSE POSITIVE"]
combined = pd.merge(planetary, stellar, on="kepid")
rstars = combined['radius'].values 
mstars = combined['mass'].values
periods = combined['koi_period'].values
prads = combined['koi_prad'].values
eccs = combined['koi_eccen'].values

In [ ]:
pgeoms = rstars / get_a(periods, mstars)

In [ ]:
int(sum(plt.hist(pgeoms[(np.isfinite(pgeoms)) & (pgeoms <= 1.0)])[0]))

In [ ]:
# these things should be an import from completeness.py
# except that this only uses pdet and not pwin

cdpp_cols = [k for k in stellar.keys() if k.startswith("rrmscdpp")]
cdpp_vals = np.array([k[-4:].replace("p", ".") for k in cdpp_cols], dtype=float)
pgam = gamma(4.65, loc=0., scale=0.98)
mesthres_cols = [k for k in stellar.keys() if k.startswith("mesthres")]
mesthres_vals = np.array([k[-4:].replace("p", ".") for k in mesthres_cols],
                         dtype=float)

def pdet_combined(catalog):
    return pcomp_vectors(catalog, catalog['koi_period'].values, 
                         catalog['koi_prad'].values, catalog['koi_eccen'].values)

def pdet_vectors(stars, periods, rp, eccs):
    '''
    Self-contained, returns pcomp over matched arrays of planets around stars.
    '''
    cdpp_cols = [k for k in stellar.keys() if k.startswith("rrmscdpp")]
    cdpp_vals = np.array([k[-4:].replace("p", ".") for k in cdpp_cols], dtype=float)
    pgam = gamma(4.65, loc=0., scale=0.98)
    mesthres_cols = [k for k in stellar.keys() if k.startswith("mesthres")]
    mesthres_vals = np.array([k[-4:].replace("p", ".") for k in mesthres_cols],
                            dtype=float)
    mstars = stars['mass'].values
    rstars = stars['radius'].values
    cdpp = stars[cdpp_cols].values
    dataspan = stars['dataspan'].values
    dutycycle = stars['dutycycle'].values
    mesthres_cols_stars = stars[mesthres_cols].values

    return pcomp_star_vectors(mstars, rstars, cdpp, dataspan, dutycycle, mesthres_cols_stars, periods, rp, eccs)

def pdet_star_vectors(mstars, rstars, cdpp, dataspan, dutycycle, mesthres_cols_stars, periods, rp, eccs):
    c = 1.0874
    s = 1.0187
    Go4pi = 2945.4625385377644/(4*np.pi*np.pi)
    re = 0.009171
    aor = (Go4pi*periods*periods*mstars) ** (1./3) / rstars
    tau = 6 * periods * np.sqrt(1 - eccs**2) / aor

    # sigma = np.apply_along_axis(np.interp, 0, tau, cdpp_vals, cdpp)
    sigma = np.array([np.interp(tau[i], cdpp_vals, cdpp[i]) for i in range(len(tau))])
    # Compute the radius ratio and estimate the S/N.
    k = rp * re / rstars
    delta = 0.84 * k*k * (c + s*k)
    snr = delta * 1e6 / sigma

    # Scale by the estimated number of transits.
    ntrn = dataspan * dutycycle / periods
    mess = snr * np.sqrt(ntrn)
    mest = np.array([np.interp(tau[i], mesthres_vals, mesthres_cols_stars[i]) for i in range(len(tau))])
    x = mess - 4.1 - (mest - 7.1)
    pdets = pgam.cdf(x)
    
    return pdets

In [ ]:
planetary_params = np.vstack((planetary.koi_period.values, 
                              planetary.koi_prad.values, planetary.koi_eccen.values)).T

In [ ]:
if not path.exists('../data/idem_pdets.npy'):
    pdet_ij = np.empty((len(planetary), len(stellar)))
    for i, params in enumerate(tqdm(planetary_params, total=len(planetary))):
        pdet_ij[i] = pcomp_vectors(stellar, *params)
    np.save('../data/idem_pdets.npy', pdet_ij)
else:
    pdet_ij = np.load('../data/idem_pdets.npy')

In [ ]:
pdet_i = np.nanmean(pdet_ij, axis=1)

In [ ]:
np.save('../data/idem_pdets_i.npy', pdet_i)

In [ ]:
plt.hist(pdet_i)

In [ ]:
weights = np.nan_to_num(1 / (pdet_i * pgeoms))

In [ ]:
period_bins = np.array([0.5, 1.25, 2.5, 5, 10, 20, 40, 80, 160, 320])
rp_bins = np.array([0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.5, 3, 4, 6, 8, 12, 16])
counts = np.histogram2d(periods, prads, bins=[period_bins, rp_bins])[0]
N = np.histogram2d(periods, prads, bins=[period_bins, rp_bins], weights=weights)
f = N[0] / len(stellar)

In [ ]:
plt.imshow(f)
plt.colorbar()
_ = plt.xticks(list(range(len(rp_bins))), rp_bins)
plt.xlabel(r"Radius ($R_E$)")
_ = plt.yticks(list(range(len(period_bins))), period_bins)
plt.ylabel("Period (days)")

In [ ]:
sigmas = np.divide(f, np.sqrt(counts), out=np.zeros_like(f), where=counts!=0)

In [ ]:
plt.imshow(sigmas)
plt.colorbar()